![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FServing&file=Serve+TensorFlow+SavedModel+Format+With+BigQuery.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FServing%2FServe%2520TensorFlow%2520SavedModel%2520Format%2520With%2520BigQuery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Serving/Serve%20TensorFlow%20SavedModel%20Format%20With%20BigQuery.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Serve TensorFlow SavedModel Format With BigQuery

Serve model predictions inside BigQuery for [TensorFlow SavedModel](https://www.tensorflow.org/guide/saved_model) format.

BigQuery has a vast set of capabilities related to ML known as BigQuery ML (or BQML for short).
- [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction)
- [BigQuery ML user journey for models](https://cloud.google.com/bigquery/docs/e2e-journey)

> For many workflow examples with BigQuery ML check out the [Framework Workflows/BQML](../../Framework%20Workflows/BQML/readme.md) folder in this repository!

With these capabilities you can train models directly in BigQuery, import model files for serving inside BigQuery, or connect to remote models for use from BigQuery.

This workflow focuses on importing a model, specifically a TensorFlow SavedModel format model as covered in [this documentation page](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow). 

There are limits to cover prior to getting started:
- not all BigQuery ML functions will work with imported models
- Models are limited to sizes under 450MB
- Model files must be in GCS, int he SavedModel format, use a GraphDef version of atleast 20, only use core TensorFlow operation (no tf.contrib operations), no RaggedTensors
- see a [full list here](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow#limitations)

What this workflow does:
- Prepare the TensorFlow SavedModel files created in this repository by another workflow: [Keras With TensorFlow Overview](../../Framework%20Workflows/Keras/Keras%20With%20TensorFlow%20Overview.ipynb)
- Setup BigQuery Dataset
- Create Models with the imported model files
- Serve predictions with `ML.PREDICT` function

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [6]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [7]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [8]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('numpy', 'numpy'),
    ('tensorflow', 'tensorflow'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [9]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [10]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [11]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [12]:
REGION = 'us-central1'
SERIES = 'mlops-serving'
EXPERIMENT = 'bigquery-tensorflow'

# gcs bucket name
GCS_BUCKET = PROJECT_ID

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = SERIES
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [15]:
# import python package
import os

import tensorflow as tf

# BigQuery
from google.cloud import bigquery

# gcs
from google.cloud import storage

Clients

In [16]:
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

# gcs client
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

Parameters:

In [17]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [18]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Model Files For Examples

The TensorFlow SavedModel files used here were created in this repository by another workflow: [Keras With JAX Overview](../../Framework%20Workflows/Keras/Keras%20With%20JAX%20Overview.ipynb).  That workflow does not need to be run because the resulting model files are within this repository.  If this notebook is being used separate from a full clone of the repository then this section will fetch the files from GitHub.

In [28]:
local_dir = '../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow'

In [29]:
if not os.path.exists(local_dir):
    print('Retrieving files...')
    local_dir = DIR
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Framework Workflows/Keras/files/keras-tf-overview/tensorflow', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Files are now in folder `{local_dir}`')
else:
    print(f'Files Found in folder `{local_dir}`')             

Files Found in folder `../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow`


In [30]:
for root, _, files in os.walk(local_dir):
    for file in files:
        print(os.path.join(root, file))

../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/saved_model.pb
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/variables/variables.index
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/variables/variables.data-00000-of-00001
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/stacked_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/stacked_model/saved_model.pb
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/stacked_model/variables/variables.index
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/stacked_model/variables/variables.data-00000-of-00001
../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/final_model/fingerprint.pb
../../Framework Workflows/Keras/files/keras-tf-overview/te

In [32]:
!gcloud storage cp -r '{local_dir}' gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/models

Copying file://../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/fingerprint.pb to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/fingerprint.pb
Copying file://../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/saved_model.pb to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/saved_model.pb
Copying file://../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/variables/variables.index to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.index
Copying file://../../Framework Workflows/Keras/files/keras-tf-overview/tensorflow/embedding_model/variables/variables.data-00000-of-00001 to gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.data-00000-of-00001
Copying file://../../Framework Workflows/Keras/files/keras-tf-overvi

In [33]:
def get_signature(uri):
    loaded_model = tf.saved_model.load(uri)
    signatures = loaded_model.signatures
    print(f"Available Signatures for: {uri}")
    for signature_name, signature_fn in signatures.items():
        print(f"\nSignature Name: {signature_name}")
        print("\nInputs:")
        if hasattr(signature_fn, 'structured_input_signature') and signature_fn.structured_input_signature:
            for input_name, input_spec in signature_fn.structured_input_signature[1].items():
                print(f"  {input_name}:")
                print(f"    dtype: {input_spec.dtype}")
                print(f"    shape: {input_spec.shape}")
        else:
            print("  No structured input signature available.")
        print("\nOutputs:")
        if hasattr(signature_fn, "structured_outputs") and signature_fn.structured_outputs:
            for output_name, output_spec in signature_fn.structured_outputs.items():
                print(f"  {output_name}:")
                print(f"    dtype: {output_spec.dtype}")
                print(f"    shape: {output_spec.shape}")
        else:
            print("  No structured output signature available.")
            
    def check_xla(signatures):
        try:
            concrete_func = signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
            graph_def = concrete_func.graph.as_graph_def()
            
            def _check_node(node):
                if "XlaCallModule" in node.op:
                    return True
                for attr in node.attr.values():
                    if attr.func:
                        func_name = attr.func.name
                        for func in graph_def.library.function:
                            if func.signature.name == func_name:
                                for func_node in func.node_def:
                                    if _check_node(func_node):
                                        return True
                return False
            
            for node in graph_def.node:
                if _check_node(node):
                    return True
            return False
        except Exception as e:
            print(f'Error checking XLA: {e}')
            return None

    has_xla = check_xla(signatures)
        
    if has_xla:
        print('XLA Detected - Conversion Required for BQML.')
    else:
        print('XLA Not Detected - Model Should Work With BQML')
            
    print('\n\n')
    
    return has_xla

In [34]:
[blob.name for blob in bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/models')]

['mlops-serving/bigquery-tensorflow/models/embedding_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/embedding_model/variables/variables.index',
 'mlops-serving/bigquery-tensorflow/models/final_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/final_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/final_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/final_model/variables/variables.index',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/fingerprint.pb',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/saved_model.pb',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/variables/variables.data-00000-of-00001',
 'mlops-serving/bigquery-tensorflow/models/stacked_model/variables/variables.index']

In [35]:
models = []
for blob in bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/models'):
    if blob.name.endswith('saved_model.pb'):
        uri = f"gs://{bucket.name}/{blob.name.split('/saved_model.pb')[0]}"
        has_xla = get_signature(uri)
        if has_xla:
            print('Model has XLA and will not work with BQML.')
        else:
            models.append(uri)

Available Signatures for: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model

Signature Name: serve

Inputs:
  keras_tensor_95:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  output_0:
    dtype: <dtype: 'float32'>
    shape: (None, 4)

Signature Name: serving_default

Inputs:
  keras_tensor_95:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  output_0:
    dtype: <dtype: 'float32'>
    shape: (None, 4)
XLA Not Detected - Model Should Work With BQML



Available Signatures for: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/final_model

Signature Name: serve

Inputs:
  input_layer:
    dtype: <dtype: 'float32'>
    shape: (None, 30)

Outputs:
  normalized_reconstruction:
    dtype: <dtype: 'float32'>
    shape: (None, 30)
  encoded:
    dtype: <dtype: 'float32'>
    shape: (None, 4)
  denormalized_reconstruction:
    dtype: <dtype: 'float32'>
    shape: (None, 30)
  normalized_RMSE:
    dtype: <dtype:

---
## BigQuery Model Import

### Create/Recall Dataset

In [36]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [37]:
job = bq.query(f"""
CREATE OR REPLACE TABLE
#CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

11.4

In [38]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,64268.0,1.316193,-0.009926,-0.137555,0.587167,-0.308887,-0.799481,0.042878,-0.080183,0.612074,...,-0.136149,-0.126894,0.639381,0.595106,-0.070506,-0.010366,0.0,0,12495373-62f2-4681-a61c-cb767b6f0117,TEST
1,6538.0,1.139435,0.666018,0.999076,2.620147,-0.194133,-0.265980,-0.103196,-0.046573,0.311083,...,0.069927,0.454127,0.307426,-0.167981,-0.036667,0.010581,0.0,0,126d9ace-632e-4f3a-877f-d2fa019c45d8,TEST
2,85326.0,1.285768,-0.051961,0.174061,-0.254305,-0.434910,-0.701094,-0.133313,-0.022129,0.125110,...,0.072654,0.024097,0.151693,0.853730,-0.097822,-0.012016,0.0,0,455897e1-27ca-449b-9e62-4248ff2dc31c,TEST
3,151032.0,-0.396177,1.625048,1.922579,4.122399,0.764063,1.739775,0.275174,0.424435,-1.666776,...,-0.137620,-0.111399,0.012638,0.098132,0.352989,0.157917,0.0,0,deec0d75-5535-4b2c-9f2f-55d2ef3a757f,TEST
4,67443.0,1.263510,0.102121,-0.005493,0.486423,-0.130838,-0.508774,0.090794,-0.066110,0.009589,...,-0.153878,0.058095,0.661726,0.559488,-0.059897,-0.014047,0.0,0,9dc517d6-aa8f-4b9d-8a05-bd7fee582b68,TEST


### Add A Column For Feature Array

Combine all the features into a single column, an array of floats, for easier inference.

In [39]:
feature_columns = [col for col in raw_sample.columns if col not in ['splits', 'transaction_id', 'Class']]
feature_columns

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount']

In [40]:
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
SELECT
    t.*, # EXCEPT(features_array),
    ARRAY[
        {', '.join(feature_columns)}
    ] AS features_array
FROM
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS t;
"""

job = bq.query(query)
job.result()
(job.ended - job.started).total_seconds()

12.389

### Review the number of records for each level of `Class` for each of the data splits:

In [41]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

,splits,class,count,percentage
0,TEST,1,50,10.16
1,TRAIN,1,393,79.88
2,VALIDATE,1,49,9.96
3,TEST,0,28432,10.00
4,TRAIN,0,227452,80.00
5,VALIDATE,0,28431,10.00


### Create The Models

In [42]:
models

['gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model',
 'gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/final_model',
 'gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/stacked_model']

In [43]:
bq_models = []
for model in models:
    bq_model = f"{BQ_PROJECT}.{BQ_DATASET}.{SERIES}-{EXPERIMENT}-{model.split('/')[-1]}"
    job = bq.query(f"""
    CREATE OR REPLACE MODEL `{bq_model}`
        OPTIONS(
            MODEL_TYPE = 'TENSORFLOW',
            MODEL_PATH = '{model}/*'
        )
    """)
    job.result()
    bq_models.append(bq_model)
    print(f"Created BigQuery Model:\n\tName: {bq_model}\n\tGCS URI: {model}\n\tTime (seconds): {(job.ended-job.started).total_seconds()}")

Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-embedding_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/embedding_model
	Time (seconds): 11.048
Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-final_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/final_model
	Time (seconds): 13.186
Created BigQuery Model:
	Name: statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-stacked_model
	GCS URI: gs://statmike-mlops-349915/mlops-serving/bigquery-tensorflow/models/stacked_model
	Time (seconds): 11.892


In [44]:
bq_models

['statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-embedding_model',
 'statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-final_model',
 'statmike-mlops-349915.mlops_serving.mlops-serving-bigquery-tensorflow-stacked_model']

### Predictions With ML.PREDICT

In [46]:
bq.query(f"""
SELECT *
FROM ML.PREDICT (MODEL `{bq_models[0]}`,(
    SELECT features_array as keras_tensor_95
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST' and class = 1
    LIMIT 10)
  )
""").to_dataframe()

,output_0,keras_tensor_95
0,"[0.0, 0.0, 0.022413315251469612, 0.00342315738...","[17230.0, -0.469326705102944, 1.11145278248281..."
1,"[0.0875319167971611, 0.0, 0.010229881852865219...","[93904.0, -11.3206331107815, 7.191950142623599..."
2,"[0.004252964165061712, 0.0, 0.0146360481157898...","[76575.0, -5.62246941906299, 3.48062317879011,..."
3,"[0.0, 0.0, 0.02239852584898472, 0.003396582556...","[48380.0, -2.79077103894771, -1.46426898705405..."
4,"[0.2401971071958542, 0.0, 0.019970163702964783...","[157207.0, 1.17075621526687, 2.50103820829712,..."
5,"[0.000820515095256269, 0.0, 0.0209587011486291...","[58222.0, -1.32278906321956, 1.55276821126964,..."
6,"[0.2842050790786743, 0.0, 0.018399760127067566...","[93879.0, -12.8336314219519, 7.50878980635543,..."
7,"[0.081075519323349, 0.0, 0.012033868581056595,...","[102676.0, -5.5521221713523605, 5.678133912586..."
8,"[0.004610396455973387, 0.0, 0.0137983886525034...","[55760.0, -6.00342167645829, -3.93073139597263..."
9,"[0.08628396689891815, 0.0, 0.01322791166603565...","[144839.0, -6.4233064564522095, 1.658514508760..."


In [48]:
bq.query(f"""
SELECT *
FROM ML.PREDICT (MODEL `{bq_models[1]}`,(
    SELECT features_array as input_layer
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST' and class = 1
    LIMIT 10)
  )
""").to_dataframe()

,denormalized_MAE,denormalized_MSE,denormalized_MSLE,denormalized_RMSE,denormalized_reconstruction,denormalized_reconstruction_errors,encoded,normalized_MAE,normalized_MSE,normalized_MSLE,normalized_RMSE,normalized_reconstruction,normalized_reconstruction_errors,input_layer
0,1563.083374,7.318019e+07,0.270502,8554.541992,"[64085.15234375, -0.3804877996444702, 0.129616...","[-46855.15234375, -0.08883890509605408, 0.9818...","[0.0, 0.0, 0.022413315251469612, 0.00342315738...",0.657085,1.072581,0.120190,1.035655,"[-0.6486759781837463, -0.20184414088726044, 0....","[-0.986429750919342, -0.04600934684276581, 0.5...","[17230.0, -0.469326705102944, 1.11145278248281..."
1,229.564148,1.495882e+06,0.594617,1223.062744,"[100602.8203125, -0.11010782420635223, 0.10655...","[-6698.8203125, -11.210525512695312, 7.0853958...","[0.0875319167971611, 0.0, 0.010229881852865219...",5.581712,71.346283,0.513062,8.446672,"[0.12012144178152084, -0.0618152916431427, 0.0...","[-0.14102862775325775, -5.805891513824463, 4.3...","[93904.0, -11.3206331107815, 7.191950142623599..."
2,426.424347,5.216996e+06,0.503197,2284.074219,"[64066.16796875, -0.38061726093292236, 0.12962...","[12508.83203125, -5.241852283477783, 3.3509967...","[0.004252964165061712, 0.0, 0.0146360481157898...",2.938603,18.778042,0.322880,4.333364,"[-0.6490756273269653, -0.20191118121147156, 0....","[0.263345330953598, -2.7147367000579834, 2.042...","[76575.0, -5.62246941906299, 3.48062317879011,..."
3,530.228638,8.220026e+06,0.262495,2867.058838,"[64082.4375, -0.38050615787506104, 0.129617974...","[-15702.4375, -2.4102649688720703, -1.59388697...","[0.0, 0.0, 0.02239852584898472, 0.003396582556...",0.692246,0.728761,0.135286,0.853675,"[-0.6487330794334412, -0.2018536478281021, 0.0...","[-0.3305795192718506, -1.248267650604248, -0.9...","[48380.0, -2.79077103894771, -1.46426898705405..."
4,1044.545410,3.260151e+07,0.329652,5709.773438,"[125933.2890625, 0.39107441902160645, 0.039849...","[31273.7109375, 0.7796818017959595, 2.46118927...","[0.2401971071958542, 0.0, 0.019970163702964783...",1.589731,5.848005,0.249205,2.418265,"[0.6533974409103394, 0.197745218873024, 0.0291...","[0.6583976745605469, 0.403794527053833, 1.4999...","[157207.0, 1.17075621526687, 2.50103820829712,..."
5,205.554947,1.152827e+06,0.443736,1073.697998,"[64097.7109375, -0.3804031014442444, 0.1296100...","[-5875.7109375, -0.9423859715461731, 1.4231581...","[0.000820515095256269, 0.0, 0.0209587011486291...",1.598017,4.700675,0.276829,2.168104,"[-0.6484115719795227, -0.20180027186870575, 0....","[-0.1236998438835144, -0.48805832862854004, 0....","[58222.0, -1.32278906321956, 1.55276821126964,..."
6,1293.982910,4.953072e+07,0.824960,7037.806152,"[132426.53125, 0.5373673439025879, 0.019872020...","[-38547.53125, -13.37099838256836, 7.488917827...","[0.2842050790786743, 0.0, 0.018399760127067566...",6.062381,69.605370,0.671706,8.342983,"[0.7900980114936829, 0.27350980043411255, 0.01...","[-0.8115315437316895, -6.924793243408203, 4.56...","[93879.0, -12.8336314219519, 7.50878980635543,..."
7,175.153061,8.498958e+05,0.708146,921.897888,"[97627.5859375, -0.13214048743247986, 0.108434...","[5048.4140625, -5.419981479644775, 5.569699764...","[0.081075519323349, 0.0, 0.012033868581056595,...",3.494400,23.353994,0.555087,4.832597,"[0.057484500110149384, -0.07322593033313751, 0...","[0.1062830314040184, -2.8069894313812256, 3.39...","[102676.0, -5.5521221713523605, 5.678133912586..."
8,288.617859,2.315222e+06,0.433717,1521.585449,"[64089.0703125, -0.38046208024024963, 0.129614...","[-8329.0703125, -5.622959613800049, -4.0603461...","[0.004610396455973387, 0.0, 0.0137983886525034...",1.404835,3.395441,0.271386,1.842672,"[-0.6485934853553772, -0.20183081924915314, 0....","[-0.17534983158111572, -2.9121110439300537, -2...","[55760.0, -6.00342167645829, -3.93073139597263..."
9,1502.083374,6.740504e+07,0.592218,8210.057617,"[99870.671875, -0.11552933603525162, 0.1070169...","[44968.328125, -6.307776927947998, 1.551497578...","[0.08628396

In [49]:
bq.query(f"""
SELECT *
FROM ML.PREDICT (MODEL `{bq_models[2]}`,(
    SELECT features_array as keras_tensor_10
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST' and class = 1
    LIMIT 10)
  )
""").to_dataframe()

,output_0,keras_tensor_10
0,"[64085.15234375, -0.3804877996444702, 0.129616...","[17230.0, -0.469326705102944, 1.11145278248281..."
1,"[100602.828125, -0.11010780185461044, 0.106554...","[93904.0, -11.3206331107815, 7.191950142623599..."
2,"[64066.16796875, -0.38061726093292236, 0.12962...","[76575.0, -5.62246941906299, 3.48062317879011,..."
3,"[64082.4375, -0.38050615787506104, 0.129617974...","[48380.0, -2.79077103894771, -1.46426898705405..."
4,"[125933.2890625, 0.39107441902160645, 0.039849...","[157207.0, 1.17075621526687, 2.50103820829712,..."
5,"[64097.7109375, -0.3804031014442444, 0.1296100...","[58222.0, -1.32278906321956, 1.55276821126964,..."
6,"[132426.53125, 0.5373673439025879, 0.019872032...","[93879.0, -12.8336314219519, 7.50878980635543,..."
7,"[97627.5859375, -0.13214048743247986, 0.108434...","[102676.0, -5.5521221713523605, 5.678133912586..."
8,"[64089.0703125, -0.38046208024024963, 0.129614...","[55760.0, -6.00342167645829, -3.93073139597263..."
9,"[99870.671875, -0.11552933603525162, 0.1070169...","[144839.0, -6.4233064564522095, 1.658514508760..."
